# DATA ENGINEERING ETL PROCESS USING PYTHON


In [2]:
from datetime import datetime
import glob
import pandas as pd 
import xml.etree.cElementTree as ET
import urllib.request
from zipfile import ZipFile


## DOWLOADING .ZIP FILE

In [ ]:
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip"

urllib.request.urlretrieve(url,"datasource.zip")

## UNZIPING FILE

In [4]:
with ZipFile("datasource.zip","r") as ZipF:
    ZipF.extractall()

## SET PATHS

In [7]:
logfile="logfile.txt" #TO STORE ALL LOG EVENTS
targetfile="transformed_data.csv" #TO STORE ALL TRANSFORMED DATA



## EXTRACT

### EXTRACT FROM CSV

In [ ]:
def extract_csv(file_to_process):
    df=pd.read_csv(file_to_process)
    return df


extract_csv("used_car_prices1.csv") #to see if the function is working

### EXTRACT FROM JSON

In [59]:


def extract_json(file_to_process):
    df=pd.read_json(file_to_process,lines=True)
    return df

#extract_json("used_car_prices1.json")#to see if the function is working


### EXTRACT FROM XML

In [60]:
def extract_xml(file_to_process):
    df=pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    tree=ET.parse(file_to_process)
    root=tree.getroot()
    for i in root:
        car_model=i.find("car_model").text
        year_of_manufacture=i.find("year_of_manufacture").text
        price=float(i.find("price").text)
        fuel=i.find("fuel").text
        df=df.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel}, ignore_index=True)
    return df

#extract_xml("used_car_prices1.xml")#to see if the function is working


### EXTRACT FUNCTION

In [ ]:
def extract():
    extracted_data=pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])

    for csv in glob.glob("*.csv"):
        extracted_data=extracted_data.append(extract_csv(csv),ignore_index=True)
    
    for json in glob.glob("*.json"):
        extracted_data=extracted_data.append(extract_json(json),ignore_index=True)
    
    for xml in glob.glob("*.xml"):
        extracted_data=extracted_data.append(extract_xml(xml),ignore_index=True)
    
    return extracted_data

#To check the extract function
#x=extract()
#print(x)


## TRANSFORM
In this case we will round the values of the column "price" to two decimal places 

In [ ]:
def transform(data):
    data.price=data.price.astype(float)
    data["price"]=round(data.price,2)

    return data

#transform(x) #To see if the function works

## LOADING

In [77]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile,index=False)

## LOGGING

In [83]:
def log(message):
    timestamp_format="%Y-%h-%d-%H:%M:%S"
    now=datetime.now()
    timestamp=now.strftime(timestamp_format)
    with open("logfile.txt","a") as lf:
        lf.write(timestamp + ", " + message + "\n")

## RUNNING THE ENTIRE ETL PROCESS

In [108]:
log("Starting ETL process")

In [ ]:
log("Starting with the Extract phase")
extracted_data=extract()
log("Extract phase concluded")

In [ ]:
log("Starting with the transform phase")
transform=transform(extracted_data)
log("Transform phase ended")


In [111]:
log("Load phase started")

In [112]:
load(targetfile,transform)
log("Load phase Ended")

In [ ]:
#pd.read_csv("transformed_data.csv") #To check if the new created csv file is correct